# Part 1: Introduction to Tensors

In [ ]:
import Base:println,+

mutable struct Tensor
    data 
end

+(a::Tensor, b::Tensor) = a.data + b.data

println(t::Tensor) = println(t.data)
    
x = Tensor([1,2,3,4,5])
print(x)

y = x + x
print(y)

In [ ]:
function workspace()
   atexit() do
       run(`$(Base.julia_cmd())`)
   end
   exit()
end

In [ ]:
workspace()

# Part 2: Introduction to Autograd

In [ ]:
import Base:println,+

mutable struct Tensor
    data
    creators
    creation_op
    grad 
    Tensor(data; creators=nothing, creation_op = nothing) = 
    new(data, creators, creation_op)
end

function backward(t::Tensor, grad)
    t.grad = grad
    
    if t.creation_op == "add"
        backward(t.creators[1], grad)
        backward(t.creators[2], grad)
    end
end

+(a::Tensor, b::Tensor) = Tensor(a.data + b.data; creators=[a,b], creation_op="add")
println(t::Tensor) = println(t.data)
println(t::Array{Tensor,1}) = println([i.data for i in t])
    
x = Tensor([1,2,3,4,5])
y = Tensor([2,2,2,2,2])

z = x + y
backward(z, Tensor([1,1,1,1,1]))

In [ ]:
println(x.grad)
println(y.grad)
println(z.creators)
println(z.creation_op)

In [ ]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a + b
f = c + d
g = e + f

backward(g, Tensor([1,1,1,1,1]))

println(a.grad)

# Part 3: Tensors That Are Used Multiple Times

In [ ]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c
f = d + e
backward(f, Tensor([1,1,1,1,1]))

b.grad.data == [2,2,2,2,2]

In [ ]:
b.grad.data

# Part 4: Upgrading Autograd to Support Multiple Tensors

In [ ]:
using Random
import Base:+,println

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        grad = Tensor(ones(size(t.data)))
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
        end
    end
end

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data .+ b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

println(t::Tensor) = println(t.data)

a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + b
e = b + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

println(b.grad.data == [2,2,2,2,2])

# Part 5: Add Support for Negation

In [2]:
using Random
import Base:+,-,println

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        if isnothing(grad)
            grad = Tensor(ones(size(t.data)))
        end
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
            if t.creation_op == "neg"
                backward(t.creators[1], -t.grad)
            end
        end
    end
end

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data .+ b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

function -(a::Tensor)
    if (a.autograd)
        return Tensor(a.data .* -1; autograd=true, creators=[a], creation_op = "neg")
    end
    return Tensor(a.data .* -1)
end


println(t::Tensor) = println(t.data)

a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + (-b)
e = (-b) + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

print(b.grad.data == [-2,-2,-2,-2,-2])

true

# Part 6: Add Support for Additional Functions

In [ ]:
using Random
import Base:+,-,println

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        if isnothing(grad)
            grad = Tensor(ones(size(t.data)))
        end
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
            
            if t.creation_op == "neg"
                backward(t.creators[1], -t.grad)
            end
            
#             if(self.creation_op == "sub"):
#                     self.creators[0].backward(Tensor(self.grad.data), self)
#                     self.creators[1].backward(Tensor(self.grad.__neg__().data), self)
        end
    end
end

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data + b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

function -(a::Tensor)
    if (a.autograd)
        return Tensor(a.data .* -1; autograd=true, creators=[a], creation_op = "neg")
    end
    return Tensor(a.data .* -1)
end

function -(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data - b.data; autograd=true, creators=[a,b], creation_op = "sub")
    end
    return Tensor(a.data-b.data)
end

function *(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data * b.data; autograd=true, creators=[a,b], creation_op = "mul")
    end
    return Tensor(a.data * b.data)
end

function sum(a::Tensor; dims=1)
    if (a.autograd && b.autograd)
        return Tensor(sum(a.data ;dims=dims); autograd=true, creators=[a], creation_op = "sum_"*string(dims))
    end
    return Tensor(sum(a.data ;dims=dims))
end

function expand(a::Tensor, dim, copies)
    trans_cmd = collect(1:length(size(a.data)))
    insert!(trans_cmd, dim, length(size(a.data)))
    old_shape = collect(size(a.data))
    new_shape = push!(old_shape, copies)
    new_data = permutedims(reshape(repeat(a.data, copies), (new_shape...)),trans_cmd)
    
    if (a.autograd)
        return Tensor(new_data; autograd=true, creators=[a], creation_op = "sum_"*string(dim))
    end
    return Tensor(new_data)
end




println(t::Tensor) = println(t.data)

a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + (-b)
e = (-b) + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

print(b.grad.data == [-2,-2,-2,-2,-2])

In [ ]:
d1.grad

In [ ]:
b1 = Tensor([2,2,2,2,2]; autograd=true)

In [ ]:
-b1

In [ ]:
b1

In [ ]:
Tensor(a.data+b.data).autograd

In [ ]:
d

In [ ]:
b

In [ ]:
d.creators[1]

In [ ]:
a.id

In [ ]:
d.id

In [ ]:
d.creators[2].grad

In [ ]:
b.id

In [ ]:
a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + (-b)
e = (-b) + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

print(b.grad.data == [-2,-2,-2,-2,-2])

In [3]:
a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

Tensor([5, 4, 3, 2, 1], true, nothing, nothing, 24863, Dict{Any,Any}(), nothing)

In [4]:
b1 = -b

Tensor([-2, -2, -2, -2, -2], true, Tensor[Tensor([2, 2, 2, 2, 2], true, nothing, nothing, 34241, Dict{Any,Any}(33980 => 1), nothing)], "neg", 33980, Dict{Any,Any}(), nothing)

In [5]:
d = a+b1

Tensor([-1, 0, 1, 2, 3], true, Tensor[Tensor([1, 2, 3, 4, 5], true, nothing, nothing, 82155, Dict{Any,Any}(57437 => 1), nothing), Tensor([-2, -2, -2, -2, -2], true, Tensor[Tensor([2, 2, 2, 2, 2], true, nothing, nothing, 34241, Dict{Any,Any}(33980 => 1), nothing)], "neg", 33980, Dict{Any,Any}(57437 => 1), nothing)], "add", 57437, Dict{Any,Any}(), nothing)

In [6]:
backward(d, Tensor([1,1,1,1,1]))

In [7]:
b.grad.data

5-element Array{Int64,1}:
 -1
 -1
 -1
 -1
 -1

In [ ]:
b1.grad

In [ ]:
d.autograd

In [ ]:
backward(b1, Tensor([1,1,1,1,1]))